In [ ]:
%run ./check_DOVS_METHODS.ipynb

In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re
import pickle
from pathlib import Path

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import copy
import itertools
import adjustText

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import Plot_Hist
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

# Analyze collected data

## AMI NonVee

In [ ]:
date_0 = '2023-09-24'
date_1 = '2023-09-30'
#-------------------------
save_dir_base = r'C:\Users\s346557\Documents\LocalData\dovs_check'
save_subdir = f"{date_0.replace('-','')}_{date_1.replace('-','')}"
#-----
base_dir     = os.path.join(save_dir_base, save_subdir)
base_dir_ami = os.path.join(save_dir_base, save_subdir, r'AllOPCOs\AMINonVee')
base_dir_ede = os.path.join(save_dir_base, save_subdir, r'AllOPCOs\EndEvents')
save_dir     = os.path.join(save_dir_base, save_subdir, r'AllOPCOs\Results')
#-----
assert(os.path.exists(base_dir_ami))
assert(os.path.exists(base_dir_ede))
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
if os.path.exists(os.path.join(base_dir, 'outg_rec_nb_to_files_dict.pkl')):
    with open(os.path.join(base_dir, 'outg_rec_nb_to_files_dict.pkl'), 'rb') as handle:
        outg_rec_nb_to_files_dict = pickle.load(handle)
else:
    #-------------------------
    paths = Utilities.find_all_paths(
        base_dir=base_dir_ami, 
        glob_pattern=r'ami_nonvee_[0-9]*.csv', 
        regex_pattern=None
    )
    paths=natsorted(paths)
    #-------------------------
    outg_rec_nbs_in_files = dict()
    for path in paths:
        assert(path not in outg_rec_nbs_in_files.keys())
        df = GenAn.read_df_from_csv(path)
        outg_rec_nbs_in_files[path] = df['OUTG_REC_NB_GPD_FOR_SQL'].unique().tolist()
    outg_rec_nb_to_files_dict = invert_file_to_outg_rec_nbs_dict(outg_rec_nbs_in_files)
    #-------------------------
    with open(os.path.join(base_dir, 'outg_rec_nb_to_files_dict.pkl'), 'wb') as handle:
        pickle.dump(outg_rec_nb_to_files_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
#-------------------------
all_outg_rec_nbs = list(outg_rec_nb_to_files_dict.keys())

In [ ]:
if os.path.exists(os.path.join(base_dir, 'outg_rec_nb_to_files_ede_dict.pkl')):
    with open(os.path.join(base_dir, 'outg_rec_nb_to_files_ede_dict.pkl'), 'rb') as handle:
        outg_rec_nb_to_files_ede_dict = pickle.load(handle)
else:
    #-------------------------
    paths_ede = Utilities.find_all_paths(
        base_dir=base_dir_ede, 
        glob_pattern=r'end_events_[0-9]*.csv', 
        regex_pattern=None
    )
    paths_ede=natsorted(paths_ede)
    #-------------------------
    outg_rec_nbs_in_files_ede = dict()
    for path in paths_ede:
        assert(path not in outg_rec_nbs_in_files_ede.keys())
        df = GenAn.read_df_from_csv(path)
        outg_rec_nbs_in_files_ede[path] = df['OUTG_REC_NB_GPD_FOR_SQL'].unique().tolist()
    outg_rec_nb_to_files_ede_dict = invert_file_to_outg_rec_nbs_dict(outg_rec_nbs_in_files_ede)
    #-------------------------
    with open(os.path.join(base_dir, 'outg_rec_nb_to_files_ede_dict.pkl'), 'wb') as handle:
        pickle.dump(outg_rec_nb_to_files_ede_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
#-------------------------
all_outg_rec_nbs_ede = list(outg_rec_nb_to_files_ede_dict.keys())

In [ ]:
#-------------------------
instvabc_slcr = DFSlicer(
    single_slicers = [
        dict(
            column='aep_derived_uom', 
            value='VOLT', 
            comparison_operator='=='
        ), 
        dict(
            column='aep_srvc_qlty_idntfr', 
            value=['INSTVA1', 'INSTVB1', 'INSTVC1'], 
            comparison_operator='isin'
        )
    ], 
    name='VOLT, INSTV(ABC)1', 
    join_single_slicers='and'
)
#-------------------------
volt_avg_slcr = DFSlicer(
    single_slicers = [
        dict(
            column='aep_derived_uom', 
            value='VOLT', 
            comparison_operator='=='
        ), 
        dict(
            column='aep_srvc_qlty_idntfr', 
            value='AVG', 
            comparison_operator='=='
        )
    ], 
    name='VOLT, AVG', 
    join_single_slicers='and'
)
#-------------------------
slicers=[instvabc_slcr, volt_avg_slcr]

In [ ]:
include_suboutg_endpt_plots=True

calculate_by_PN = True
# combine_by_PN_likeness_thresh = pd.Timedelta('1 minutes')
combine_by_PN_likeness_thresh = pd.Timedelta('15 minutes')

expand_outg_search_time_tight = pd.Timedelta('1 hours')
expand_outg_search_time_loose = pd.Timedelta('12 hours')
use_est_outg_times=False
# use_est_outg_times=True
use_full_ede_outgs=False
run_outg_inclusion_assessment=True
max_pct_PNs_missing_allowed=0
# max_pct_PNs_missing_allowed=20
#-----
expand_outg_est_search_time = expand_outg_search_time_loose
if use_est_outg_times:
    expand_outg_search_time = expand_outg_search_time_tight
else:
    expand_outg_search_time = expand_outg_search_time_loose

In [ ]:
pdf_path = os.path.join(save_dir, r'Results.pdf')
pdf = PdfPages(pdf_path)
#-------------------------
pdf_path_dovs_beg = Utilities.append_to_path(
    pdf_path, 
    '_dovs_beg', 
    ext_to_find='.pdf', 
    append_to_end_if_ext_no_found=False
)
pdf_dovs_beg = PdfPages(pdf_path_dovs_beg)
#-------------------------
if include_suboutg_endpt_plots:
    pdf_path_2 = Utilities.append_to_path(
        pdf_path, 
        '_w_suboutg_endpt_plots', 
        ext_to_find='.pdf', 
        append_to_end_if_ext_no_found=False
    )
    pdf_2 = PdfPages(pdf_path_2)
#-------------------------
pdf_path_n_w_power = os.path.join(save_dir, r'n_w_power_v_time.pdf')
pdf_n_w_power = PdfPages(pdf_path_n_w_power)
#-----
pdf_path_n_w_power_dovs_beg = Utilities.append_to_path(
    pdf_path_n_w_power, 
    '_dovs_beg', 
    ext_to_find='.pdf', 
    append_to_end_if_ext_no_found=False
)
pdf_n_w_power_dovs_beg = PdfPages(pdf_path_n_w_power_dovs_beg) 

In [ ]:
n_PNs_w_power_threshold = 95

In [ ]:
fig_num=0
all_detailed_summary_dfs=[]
all_detailed_summary_dfs_dovs_beg=[]
ci_cmi_summary_df = pd.DataFrame(columns=[
    'outg_rec_nb', 
    'ci_dovs',  'ci_ami', 'ci_ami_dovs_beg', 
    'cmi_dovs', 'cmi_ami', 'cmi_ami_dovs_beg'
])

#-------------------------
# Build dovs_df
dovs = DOVSOutages(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True,
    build_sql_function=DOVSOutages_SQL.build_sql_std_outage, 
    build_sql_function_kwargs=dict(
        outg_rec_nbs=all_outg_rec_nbs, 
        field_to_split='outg_rec_nbs', 
        include_premise=True
    ), 
    build_consolidated=True
)
dovs_df = dovs.df.copy()

#-------------------------
# Now, iterate through all outages
for i_outg, outg_rec_nb in enumerate(all_outg_rec_nbs):
    print(f'\n\ti_outg: {i_outg+1}/{len(all_outg_rec_nbs)}')
    print(f'\toutg_rec_nb = {outg_rec_nb}')
    #-----
    ami_df = GenAn.read_df_from_csv_batch(outg_rec_nb_to_files_dict[outg_rec_nb])
    #--------------------------------------------------
    ami_df_i = ami_df[ami_df['OUTG_REC_NB_GPD_FOR_SQL']==outg_rec_nb].copy()
    
    # Although I cannot yet call choose_best_slicer_and_perform_slicing and reduce_INSTV_ABC_1_vals_in_df, 
    #   as the standard cleaning and conversions must be done first, I am able to cut down the size of
    #   ami_df_i by joining the slicers with 'or' statements.
    # Thus, ami_df_i will be reduced to only the subset of data which will be considered in 
    #   choose_best_slicer_and_perform_slicing
    # As mentioned, this will cut down the size of ami_df_i and will also save time and resources by not having
    #   to run entire DF through cleaning and conversions procedures.
    ami_df_i = DFSlicer.combine_slicers_and_perform_slicing(
        df=ami_df_i, 
        slicers=slicers, 
        join_slicers='or'
    )
    if ami_df_i.shape[0]==0:
        continue
        
    #--------------------------------------------------
    ami_df_i = AMINonVee.perform_std_initiation_and_cleaning(ami_df_i)
    #-----
    # Should the following be added to AMINonVee.perform_std_initiation_and_cleaning?
    ami_df_i = Utilities_dt.strip_tz_info_and_convert_to_dt(
        df=ami_df_i, 
        time_col='starttimeperiod', 
        placement_col='starttimeperiod_local', 
        run_quick=True, 
        n_strip=6, 
        inplace=False
    )
    ami_df_i = Utilities_dt.strip_tz_info_and_convert_to_dt(
        df=ami_df_i, 
        time_col='endtimeperiod', 
        placement_col='endtimeperiod_local', 
        run_quick=True, 
        n_strip=6, 
        inplace=False
    )
    #--------------------------------------------------
    ami_df_i = choose_best_slicer_and_perform_slicing(
        df=ami_df_i, 
        slicers=slicers, 
        groupby_SN=True, 
        t_search_min_max=None, 
        time_col='starttimeperiod_local', 
        value_col=None, 
        SN_col='serialnumber', 
        return_sorted=True
    )

    ami_df_i = reduce_INSTV_ABC_1_vals_in_df(
        df=ami_df_i, 
        value_col='value', 
        aep_derived_uom_col='aep_derived_uom', 
        aep_srvc_qlty_idntfr_col='aep_srvc_qlty_idntfr', 
        output_aep_srvc_qlty_idntfr = 'INSTV(ABC)1'
    )

    if ami_df_i.shape[0]==0:
        continue
        
    #-------------------------
    # Each serial number should have a single value per time stamp
    assert(ami_df_i.groupby(['serialnumber', 'starttimeperiod_local']).ngroups == ami_df_i.shape[0])

    #-------------------------
    if run_outg_inclusion_assessment:
        to_include_i = assess_outage_inclusion_requirements(
            ami_df_i=ami_df_i, 
            outg_rec_nb=outg_rec_nb, 
            dovs_df=dovs_df, 
            max_pct_PNs_missing_allowed=max_pct_PNs_missing_allowed

        )
        if not to_include_i:
            print(f'outg_rec_nb={outg_rec_nb} did not pass inclusion requirements, skipping!!!!!')
            continue
    #-------------------------    
    n_SNs  = ami_df_i['serialnumber'].nunique()
    n_PNs  = ami_df_i['aep_premise_nb'].nunique()
    
    #----------------------------------------------------------------------------------------------------
    # NOTE: Can save time by grabbing ede_df_i then performing tz conversion and adding DOVS
    if outg_rec_nb not in outg_rec_nb_to_files_ede_dict.keys():
        ede_df_i=None
    else:
        ede_df = GenAn.read_df_from_csv_batch(outg_rec_nb_to_files_ede_dict[outg_rec_nb])
        ede_df_i = ede_df[ede_df['OUTG_REC_NB_GPD_FOR_SQL']==outg_rec_nb].copy()

        #-----
        ede_df_i = Utilities_dt.strip_tz_info_and_convert_to_dt(
            df=ede_df_i, 
            time_col='valuesinterval', 
            placement_col='valuesinterval_local', 
            run_quick=True, 
            n_strip=6, 
            inplace=False
        )
        ede_df_i = AMIEndEvents.reduce_end_event_reasons_in_df(df=ede_df_i)
        #-----
        ede_cols_to_keep = [
            'valuesinterval_local', 
            'reason', 
            'serialnumber', 
            'aep_premise_nb', 
            'enddeviceeventtypeid', 
            'event_type', 
            'OUTG_REC_NB_GPD_FOR_SQL', 
            'trsf_pole_nb_GPD_FOR_SQL',
        ]
        ede_df_i = ede_df_i[ede_cols_to_keep]
        
    #----------------------------------------------------------------------------------------------------
    dovs_df_i = DOVSOutages.retrieve_outage_from_dovs_df(
        dovs_df=dovs_df, 
        outg_rec_nb=outg_rec_nb, 
        outg_rec_nb_idfr='index', 
        assert_outg_rec_nb_found=True
    )
    assert(dovs_df_i.shape[0]==1)
    # Get the outage time from DOVS
    dovs_outg_t_beg_end = dovs_df_i.iloc[0][['DT_OFF_TS_FULL', 'DT_ON_TS']].tolist()
    assert(len(dovs_outg_t_beg_end)==2)
    dovs_outg_t_beg, dovs_outg_t_end = dovs_outg_t_beg_end
    #-------------------------
    # Get the CI and CMI from DOVS
    ci_cmi_dovs = dovs_df_i.iloc[0][['CI_NB', 'CMI_NB']].tolist()
    assert(len(ci_cmi_dovs)==2)
    ci_dovs, cmi_dovs = ci_cmi_dovs
    #-------------------------
    # Get the number of premises from DOVS
    n_PNs_dovs = len(set(dovs_df_i.iloc[0]['premise_nbs']))
    #-------------------------
    # Get the outage number from DOVS
    outage_nb = dovs_df_i.iloc[0]['OUTAGE_NB']
    
    #--------------------------------------------------
    res_dict = calculate_ci_cmi_w_ami_w_ede_help(
        df=ami_df_i, 
        ede_df=ede_df_i, 
        dovs_outg_t_beg_end=dovs_outg_t_beg_end, 
        expand_outg_search_time=expand_outg_search_time, 
        conservative_estimate=True, 
        est_ede_kwargs=dict(use_full_ede_outgs=use_full_ede_outgs), 
        audit_selection_method='ede only', 
        return_CI_SNs=False, 
        use_est_outg_times=use_est_outg_times, 
        pct_SNs_required_for_outage_est=0, 
        expand_outg_est_search_time=expand_outg_est_search_time, 
        use_only_overall_endpoints_of_est_outg_times=False, 
        t_int_beg_col='starttimeperiod_local', 
        t_int_end_col='endtimeperiod_local', 
        value_col='value', 
        SN_col='serialnumber', 
        return_all_best_ests=True, 
        return_all_best_ests_type='pd.DataFrame'
    )
    #-----
    ci_ami       = res_dict['CI']
    cmi_ami      = res_dict['CMI']
    best_ests_df = res_dict['all_best_ests']
    #--------------------------------------------------
    if calculate_by_PN and best_ests_df.shape[0]>0:
        try:
            best_ests_df = combine_PNs_in_best_ests_df(
                best_ests_df, 
                likeness_thresh = combine_by_PN_likeness_thresh, 
                SN_col = 'SN', 
                PN_col = 'PN', 
                i_outg_col = 'i_outg'     
            )
            ci_ami  = best_ests_df['PN'].nunique()
            cmi_ami = (best_ests_df['winner_max']-best_ests_df['winner_min']).sum().total_seconds()/60
        except:
            print(f'outg_rec_nb={outg_rec_nb} failed combine_PNs_in_best_ests_df, so skipping')
            continue
    #--------------------------------------------------
    # cnsrvtv_out_t_beg/_end are used for placing bounds on the plots generated
    # If best_ests_df has non-zero size (meaning the algorithm found outages), use to set plotting time.
    # Otherwise, use dovs_outg_t_beg/_end
    # NOTE: If one did not want to show any data which was thrown out due to overlapping, one would want to
    #         update cnsrvtv_out_t_beg/_end after the identify_dovs_overlaps_from_best_ests procedure below 
    #         (and subsequent trimming of best_ests_df)
    if best_ests_df.shape[0]>0:
        cnsrvtv_out_t_beg = np.min([best_ests_df['conservative_min'].min(), dovs_outg_t_beg])
        cnsrvtv_out_t_end = np.max([best_ests_df['conservative_max'].max(), dovs_outg_t_end])
    else:
        cnsrvtv_out_t_beg = dovs_outg_t_beg
        cnsrvtv_out_t_end = dovs_outg_t_end
    #--------------------------------------------------
    if best_ests_df.shape[0]>0:
        # Identify and handle any overlaps with other DOVS events
        #-------------------------
        # dovs_sql_fcn=DOVSOutages_SQL.build_sql_std_outage
        dovs_sql_fcn=DOVSOutages_SQL.build_sql_outage        
        #-------------------------
        # I'll need potential overlapping DOVS df using (dovs_outg_t_beg, dovs_outg_t_end) and also
        #   using (best_ests_df[t_min_col].min(), best_ests_df[t_max_col].max())
        # Since these typically have significant overlap, grab potential overlapping DOVS events for 
        #   all, and then subset as needed
        ptntl_ovrlp_dovs_df = get_potential_overlapping_dovs(
            PNs=ami_df_i['aep_premise_nb'].unique().tolist(), 
            outg_t_beg=np.min([dovs_outg_t_beg, best_ests_df['winner_min'].min()]), 
            outg_t_end=np.max([dovs_outg_t_end, best_ests_df['winner_max'].max()]), 
            dovs_sql_fcn=dovs_sql_fcn, 
        )
        #-----
        ptntl_ovrlp_dovs_df_out_times = ptntl_ovrlp_dovs_df[
            (ptntl_ovrlp_dovs_df['DT_OFF_TS_FULL'] <= dovs_outg_t_end) & 
            (ptntl_ovrlp_dovs_df['DT_ON_TS']       >= dovs_outg_t_beg)
        ]
        ptntl_ovrlp_dovs_df_est_times = ptntl_ovrlp_dovs_df[
            (ptntl_ovrlp_dovs_df['DT_OFF_TS_FULL'] <= best_ests_df['winner_max'].max()) & 
            (ptntl_ovrlp_dovs_df['DT_ON_TS']       >= best_ests_df['winner_min'].min())
        ]        
        #-------------------------
        # First, find if any other DOVS outages overlap with the current one
        #-----
        # NOTE: Supply best_ests_df to get_outgs_in_dovs_df_overlapping_outg_rec_nb_i_by_PN below so 'lost_power'
        #       column is output
        overlap_outgs_for_PNs_df = get_outgs_in_dovs_df_overlapping_outg_rec_nb_i_by_PN(
            outg_rec_nb_i=outg_rec_nb, 
            dovs_df=ptntl_ovrlp_dovs_df_out_times, 
            best_ests_df=best_ests_df, 
            outg_rec_nb_col='OUTG_REC_NB', 
            PN_col='PREMISE_NB', 
            t_min_col='DT_OFF_TS_FULL', 
            t_max_col='DT_ON_TS', 
            PN_col_best_ests='PN'
        )
        #-------------------------
        # If any PN has one or more overlapping DOVS events, output info to file
        n_PNs_w_overlap = (overlap_outgs_for_PNs_df['n_overlap']>0).sum()
        if n_PNs_w_overlap>0:
            print(f'Need to output to new file\n\tn_PNs_w_overlap={n_PNs_w_overlap}')
        #-------------------------
        # If any PN which lost power has one or more overlapping DOVS events, stop analysis
        n_out_PNs_w_overlap = overlap_outgs_for_PNs_df[overlap_outgs_for_PNs_df['lost_power']==True]['overlap_outg_rec_nbs'].apply(lambda x: len(x)>0).sum()
        if n_out_PNs_w_overlap>0:
            print('STOP analysis')
            continue
        #-------------------------
        # Procedure above ensure that the current outage, as defined by DOVS, does not overlap with any other DOVS outages.
        # Now, we are safe to check whether any of our estimates for the current outage overlap with any other DOVS outages!
        # ==> Should be safe to set assert_no_overlaps=True in identify_dovs_overlaps_from_best_ests or 
        #       remove_any_dovs_overlaps_from_best_ests below
        best_ests_df_w_keep_info = identify_dovs_overlaps_from_best_ests(
            best_ests_df                = best_ests_df, 
            outg_rec_nb                 = outg_rec_nb, 
            dovs_df                     = ptntl_ovrlp_dovs_df_est_times, 
            get_ptntl_ovrlp_dovs_kwargs = None, 
            assert_no_overlaps          = True, 
            PN_col                      = 'PN', 
            t_min_col                   = 'winner_min', 
            t_max_col                   = 'winner_max', 
            PN_col_dovs                 = 'PREMISE_NB', 
            t_min_col_dovs              = 'DT_OFF_TS_FULL', 
            t_max_col_dovs              = 'DT_ON_TS', 
            outg_rec_nb_col_dovs        = 'OUTG_REC_NB', 
            overlap_outg_col            = 'overlap_DOVS', 
            overlap_times_col           = 'overlap_times', 
            keep_col                    = 'keep'
        )
        best_ests_df = best_ests_df_w_keep_info[best_ests_df_w_keep_info['keep']==True].drop(
            columns=['overlap_DOVS', 'overlap_times', 'keep']
        )
        #-----
        ci_ami  = best_ests_df['PN'].nunique()
        cmi_ami = (best_ests_df['winner_max']-best_ests_df['winner_min']).sum().total_seconds()/60
        #-------------------------
        # In ami_df_i, mark any entries which were essentially removed via the identify_dovs_overlaps_from_best_ests
        #   and removal procedure above
        ami_df_i = set_removed_due_to_overlap_in_ami_df_i(
            ami_df_i=ami_df_i, 
            best_ests_df=best_ests_df_w_keep_info, 
            PN_col='aep_premise_nb', 
            time_idfr='starttimeperiod_local', 
            PN_col_be='PN', 
            keep_col_be='keep', 
            overlap_times_col_be='overlap_times', 
            removed_due_to_overlap_col='removed_due_to_overlap'
        )
            
    #--------------------------------------------------
    if best_ests_df.shape[0]>0:
        best_ests_df_dovs_beg = alter_best_ests_df_using_dovs_outg_t_beg(
            best_ests_df=best_ests_df,
            dovs_df=dovs_df_i, 
            outg_rec_nb=outg_rec_nb
        )
        if calculate_by_PN:
            ci_ami_dovs_beg  = best_ests_df_dovs_beg['PN'].nunique()
        else:
            ci_ami_dovs_beg  = best_ests_df_dovs_beg['SN'].nunique()
        cmi_ami_dovs_beg = (best_ests_df_dovs_beg['winner_max']-best_ests_df_dovs_beg['winner_min']).sum().total_seconds()/60
    else:
        best_ests_df_dovs_beg = best_ests_df.copy()
        ci_ami_dovs_beg = ci_ami
        cmi_ami_dovs_beg = cmi_ami        
    #--------------------------------------------------
    if best_ests_df.shape[0]>0:
        means_df, best_ests_df_w_db_lbl = get_mean_times_w_dbscan(
            best_ests_df, 
            eps_min=5, 
            min_samples=2, 
            ests_to_include_in_clustering=['winner_min', 'winner_max'],
            ests_to_include_in_output=[
                'winner_min', 'winner_max', 
                'conservative_min', 'conservative_max', 
                'zero_times_min', 'zero_times_max'
            ], 
            return_labelled_best_ests_df=True
        )
        #-------------------------
        n_PNs_w_power_srs = build_n_PNs_w_power_srs(
            best_ests_df=best_ests_df, 
            ami_df_i=ami_df_i, 
            return_pct=True, 
            PN_col='PN', 
            t_min_col='winner_min', 
            t_max_col='winner_max', 
            i_outg_col='i_outg', 
            PN_col_ami_df='aep_premise_nb'
        )        
        #-------------------------
        detailed_summary_df_i = build_detailed_summary_df(
            means_df=means_df, 
            best_ests_df_w_db_lbl=best_ests_df_w_db_lbl,
            CI_tot=ci_ami, 
            CMI_tot=cmi_ami, 
            n_PNs_ami=n_PNs,
            outg_rec_nb=outg_rec_nb, 
            dovs_df_i=dovs_df_i, 
            db_label_col='db_label', 
            winner_min_col='winner_min', 
            winner_max_col='winner_max', 
            PN_col='PN' if calculate_by_PN else 'SN', 
            i_outg_col='i_outg'
        )
        #----------
        detailed_summary_df_i[f'first_above_thresh ({n_PNs_w_power_threshold})'] = None
        detailed_summary_df_i[f'last_above_thresh ({n_PNs_w_power_threshold})']  = None
        frst_abv, last_abv = get_first_last_above_threshold(
            n_PNs_w_power_srs=n_PNs_w_power_srs, 
            threshold=n_PNs_w_power_threshold
        )
        #-----
        detailed_summary_df_i.iloc[
            0, 
            detailed_summary_df_i.columns.tolist().index(f'first_above_thresh ({n_PNs_w_power_threshold})')
        ] = frst_abv
        #-----
        detailed_summary_df_i.iloc[
            0, 
            detailed_summary_df_i.columns.tolist().index(f'last_above_thresh ({n_PNs_w_power_threshold})')
        ] = last_abv        
        #-------------------------
        all_detailed_summary_dfs.append(detailed_summary_df_i)
    else:
        means_df, best_ests_df_w_db_lbl = None, None
    #--------------------------------------------------
    if best_ests_df_dovs_beg.shape[0]>0:
        means_df_dovs_beg, best_ests_df_dovs_beg_w_db_lbl = get_mean_times_w_dbscan(
            best_ests_df_dovs_beg, 
            eps_min=5, 
            min_samples=2, 
            ests_to_include_in_clustering=['winner_min', 'winner_max'],
            ests_to_include_in_output=[
                'winner_min', 'winner_max', 
                'conservative_min', 'conservative_max', 
                'zero_times_min', 'zero_times_max'
            ], 
            return_labelled_best_ests_df=True
        )
        #-----
        detailed_summary_df_dovs_beg_i = build_detailed_summary_df(
            means_df=means_df_dovs_beg, 
            best_ests_df_w_db_lbl=best_ests_df_dovs_beg_w_db_lbl,
            CI_tot=ci_ami_dovs_beg, 
            CMI_tot=cmi_ami_dovs_beg, 
            n_PNs_ami=n_PNs,
            outg_rec_nb=outg_rec_nb, 
            dovs_df_i=dovs_df_i, 
            db_label_col='db_label', 
            winner_min_col='winner_min', 
            winner_max_col='winner_max', 
            PN_col='PN' if calculate_by_PN else 'SN', 
            i_outg_col='i_outg'
        )
        all_detailed_summary_dfs_dovs_beg.append(detailed_summary_df_dovs_beg_i)
    else:
        means_df_dovs_beg, best_ests_df_dovs_beg_w_db_lbl = None, None
    #-------------------------
    ci_cmi_summary_df = pd.concat([
        ci_cmi_summary_df, 
        pd.DataFrame(
            dict(
                outg_rec_nb=outg_rec_nb, 
                ci_dovs=ci_dovs,   ci_ami=ci_ami, ci_ami_dovs_beg=ci_ami_dovs_beg, 
                cmi_dovs=cmi_dovs, cmi_ami=cmi_ami, cmi_ami_dovs_beg=cmi_ami_dovs_beg
            ), 
            index=[ci_cmi_summary_df.shape[0]]
        )
    ])
    #----------------------------------------------------------------------------------------------------
    # ######################### PLOTTING #########################
    #----------------------------------------------------------------------------------------------------
    #--------------------------------------------------
    # Instead of using get_full_part_not_outage_subset_dfs, simply grab the PNs which suffered
    #   outages from best_ests_df
    if best_ests_df.shape[0]>0:
        outg_SNs = best_ests_df['PN'].unique().tolist()
    else:
        outg_SNs = []
    #-----
    ami_df_i_out      = ami_df_i[ami_df_i['aep_premise_nb'].isin(outg_SNs)]
    ami_df_i_not_out  = ami_df_i[~ami_df_i['aep_premise_nb'].isin(outg_SNs)]  

    #----------------------------------------------------------------------------------------------------
    fig, axs = plot_all_out_not_NEW(
        fig_num              = fig_num, 
        ami_df_i             = ami_df_i, 
        ami_df_i_out         = ami_df_i_out, 
        ami_df_i_not_out     = ami_df_i_not_out, 
        dovs_outg_t_beg      = dovs_outg_t_beg, 
        dovs_outg_t_end      = dovs_outg_t_end, 
        cnsrvtv_out_t_beg    = cnsrvtv_out_t_beg, 
        cnsrvtv_out_t_end    = cnsrvtv_out_t_end, 
        means_df             = means_df, 
        outg_rec_nb          = outg_rec_nb, 
        outage_nb            = outage_nb, 
        n_PNs_dovs           = n_PNs_dovs, 
        ci_dovs              = ci_dovs, 
        cmi_dovs             = cmi_dovs, 
        ci_ami               = ci_ami, 
        cmi_ami              = cmi_ami, 
        name                 = 'AMI', 
        results_2_dict       = dict(
            ci_ami   = ci_ami_dovs_beg, 
            cmi_ami  = cmi_ami_dovs_beg, 
            means_df = means_df_dovs_beg, 
            name = 'AMI w/ DOVS t_beg'
        ), 
        expand_time          = pd.Timedelta('1 hour'), 
        removed_due_to_overlap_col='removed_due_to_overlap', 
        mean_keys_to_include = ['winner', 'conservative', 'zero_times'], 
        default_subplots_args=dict(n_x=2, n_y=2, row_major=True, sharex=True), 
        leg_i_plot=1, 
        leg_kwargs=dict(ncols=1, fontsize=15, bbox_to_anchor=(1, 1.2)), 
        ci_info_fontsize=16, 
        left_text_x=0.915  
    )

    fig, axs[3] = plot_n_PNs_w_power_srs(
        n_PNs_w_power_srs=n_PNs_w_power_srs, 
        simp_freq='1T', 
        threshold=n_PNs_w_power_threshold, 
        fig_num=fig_num, 
        fig_ax=(fig, axs[3]), 
        threshold_color='magenta'
    )

    fig_num += 1
    pdf.savefig(fig, bbox_inches='tight')
    if include_suboutg_endpt_plots:
        pdf_2.savefig(fig, bbox_inches='tight')
    plt.close(fig)
    
    #----------------------------------------------------------------------------------------------------
    if means_df_dovs_beg is not None:
        fig, axs = plot_all_out_not_NEW(
            fig_num              = fig_num, 
            ami_df_i             = ami_df_i, 
            ami_df_i_out         = ami_df_i_out, 
            ami_df_i_not_out     = ami_df_i_not_out, 
            dovs_outg_t_beg      = dovs_outg_t_beg, 
            dovs_outg_t_end      = dovs_outg_t_end, 
            cnsrvtv_out_t_beg    = cnsrvtv_out_t_beg, 
            cnsrvtv_out_t_end    = cnsrvtv_out_t_end, 
            means_df             = means_df_dovs_beg, 
            outg_rec_nb          = outg_rec_nb, 
            outage_nb            = outage_nb, 
            n_PNs_dovs           = n_PNs_dovs, 
            ci_dovs              = ci_dovs, 
            cmi_dovs             = cmi_dovs, 
            ci_ami               = ci_ami_dovs_beg, 
            cmi_ami              = cmi_ami_dovs_beg, 
            name                 = 'AMI w/ DOVS t_beg', 
            results_2_dict       = dict(
                ci_ami   = ci_ami, 
                cmi_ami  = cmi_ami, 
                means_df = means_df, 
                name = 'AMI'
            ), 
            expand_time          = pd.Timedelta('1 hour'), 
            mean_keys_to_include = ['winner']
        )

        fig_num += 1
        pdf_dovs_beg.savefig(fig, bbox_inches='tight')
        plt.close(fig)
    
    #----------------------------------------------------------------------------------------------------
    if means_df is not None and include_suboutg_endpt_plots:
        fig, axs = plot_suboutg_endpts(
            fig_num               = fig_num, 
            ami_df_i              = ami_df_i, 
            means_df              = means_df, 
            best_ests_df_w_db_lbl = best_ests_df_w_db_lbl, 
            dovs_outg_t_beg       = dovs_outg_t_beg, 
            dovs_outg_t_end       = dovs_outg_t_end, 
            outg_rec_nb           = outg_rec_nb, 
            expand_time           = pd.Timedelta('15 minutes'), 
            mean_keys_to_include  = ['winner', 'conservative', 'zero_times']
        )
        #-------------------------
        fig_num += 1
        pdf_2.savefig(fig, bbox_inches='tight')     
        plt.close(fig)
        
    #----------------------------------------------------------------------------------------------------
    if best_ests_df.shape[0]>0:
        _, fig, ax = build_n_PNs_w_power_srs_and_plot(
            best_ests_df, 
            ami_df_i, 
            return_pct=True, 
            simp_freq='1T', 
            threshold=n_PNs_w_power_threshold, 
            fig_num=fig_num, 
            title=f"OUTG_REC_NB = {outg_rec_nb}", 
            PN_col='PN', 
            t_min_col='winner_min', 
            t_max_col='winner_max', 
            i_outg_col='i_outg', 
            PN_col_ami_df='aep_premise_nb'
        )
        #-------------------------
        fig_num += 1
        pdf_n_w_power.savefig(fig, bbox_inches='tight')     
        plt.close(fig)
    #----------------------------------------------------------------------------------------------------
    if best_ests_df_dovs_beg.shape[0]>0:
        _, fig, ax = build_n_PNs_w_power_srs_and_plot(
            best_ests_df_dovs_beg, 
            ami_df_i, 
            return_pct=True, 
            simp_freq='1T', 
            threshold=n_PNs_w_power_threshold, 
            fig_num=fig_num, 
            title=f"OUTG_REC_NB = {outg_rec_nb}", 
            PN_col='PN', 
            t_min_col='winner_min', 
            t_max_col='winner_max', 
            i_outg_col='i_outg', 
            PN_col_ami_df='aep_premise_nb'
        )
        #-------------------------
        fig_num += 1
        pdf_n_w_power_dovs_beg.savefig(fig, bbox_inches='tight')     
        plt.close(fig)
        
#----------------------------------------------------------------------------------------------------
detailed_summary_df          = pd.concat(all_detailed_summary_dfs)
detailed_summary_df_dovs_beg = pd.concat(all_detailed_summary_dfs_dovs_beg)

In [ ]:
pdf.close()
pdf_dovs_beg.close()
if include_suboutg_endpt_plots:
    pdf_2.close()
pdf_n_w_power.close()
pdf_n_w_power_dovs_beg.close()

In [ ]:
ci_cmi_summary_df['ci_dovs']         = ci_cmi_summary_df['ci_dovs'].astype(float)
ci_cmi_summary_df['ci_ami']          = ci_cmi_summary_df['ci_ami'].astype(float)
ci_cmi_summary_df['ci_ami_dovs_beg'] = ci_cmi_summary_df['ci_ami_dovs_beg'].astype(float)
#-----
ci_cmi_summary_df['delta_ci_dovs_ami']  = ci_cmi_summary_df['ci_dovs']-ci_cmi_summary_df['ci_ami']
ci_cmi_summary_df['delta_cmi_dovs_ami'] = ci_cmi_summary_df['cmi_dovs']-ci_cmi_summary_df['cmi_ami']
#-----
ci_cmi_summary_df['delta_ci_dovs_ami_dovs_beg']  = ci_cmi_summary_df['ci_dovs']-ci_cmi_summary_df['ci_ami_dovs_beg']
ci_cmi_summary_df['delta_cmi_dovs_ami_dovs_beg'] = ci_cmi_summary_df['cmi_dovs']-ci_cmi_summary_df['cmi_ami_dovs_beg']
#-----
# For plotting purposes, make a outg_rec_in column which is simply 0 to delta_df.shape[0]-1
ci_cmi_summary_df['outg_rec_int'] = range(ci_cmi_summary_df.shape[0])
#-----

In [ ]:
detailed_summary_df.to_pickle(os.path.join(save_dir, r'detailed_summary.pkl'))
detailed_summary_df_dovs_beg.to_pickle(os.path.join(save_dir, r'detailed_summary_dovs_beg.pkl'))
ci_cmi_summary_df.to_pickle(os.path.join(save_dir, r'ci_cmi_summary.pkl'))
#-----
detailed_summary_df.to_csv(os.path.join(save_dir, r'detailed_summary.csv'))
detailed_summary_df_dovs_beg.to_csv(os.path.join(save_dir, r'detailed_summary_dovs_beg.csv'))
ci_cmi_summary_df.to_csv(os.path.join(save_dir, r'ci_cmi_summary.csv'))
#-----
# For Mico and Amanda
detailed_summary_df_dovs_beg.to_csv(os.path.join(save_dir, f'detailed_summary_dovs_beg_{save_subdir}.csv'))

In [ ]:
assert(0)

# ===========================================================